<a href="https://colab.research.google.com/github/Cincin3112/Cross-lingual-NLI-for-Thai/blob/main/mBERT_Cross_Lingual_Thai_NLI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install & import

In [1]:
#check GPU 
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Dec  7 13:59:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    30W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.9 MB/s 
     |████████████████████████████████| 182 kB 73.1 MB/s 
     |████████████████████████████████| 7.6 MB 54.4 MB/s 


In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.5 MB/s 
     |████████████████████████████████| 132 kB 70.1 MB/s 
     |████████████████████████████████| 212 kB 67.4 MB/s 
     |████████████████████████████████| 127 kB 72.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [4]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.0 MB/s 


In [5]:
from transformers import (
    AutoTokenizer, 
    AutoModel,
    AutoModelForSequenceClassification, 
    AutoConfig,
    Trainer, 
    TrainingArguments,
)
from datasets import (
    Dataset,
    DatasetDict,
    load_dataset,
    Features, Sequence, ClassLabel, Value
)

import pandas as pd

# load model

In [6]:
model_name = 'bert-base-multilingual-cased'
config = AutoConfig.from_pretrained(model_name, num_labels=3)
model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [7]:
from datasets import list_metrics, load_metric
list_metrics()

<ipython-input-7-3cf64b9e62a1>:2: FutureWarning: list_metrics is deprecated and will be removed in the next major version of datasets. Use 'evaluate.list_evaluation_modules' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  list_metrics()


['accuracy',
 'bertscore',
 'bleu',
 'bleurt',
 'brier_score',
 'cer',
 'chrf',
 'code_eval',
 'comet',
 'competition_math',
 'coval',
 'cuad',
 'exact_match',
 'f1',
 'frugalscore',
 'glue',
 'google_bleu',
 'indic_glue',
 'mae',
 'mahalanobis',
 'mape',
 'mase',
 'matthews_correlation',
 'mauve',
 'mean_iou',
 'meteor',
 'mse',
 'nist_mt',
 'pearsonr',
 'perplexity',
 'poseval',
 'precision',
 'recall',
 'rl_reliability',
 'roc_auc',
 'rouge',
 'sacrebleu',
 'sari',
 'seqeval',
 'smape',
 'spearmanr',
 'squad',
 'squad_v2',
 'super_glue',
 'ter',
 'trec_eval',
 'wer',
 'wiki_split',
 'xnli',
 'xtreme_s',
 'Drunper/metrica_tesi',
 'Felipehonorato/my_metric',
 'GMFTBY/dailydialog_evaluate',
 'GMFTBY/dailydialogevaluate',
 'JP-SystemsX/nDCG',
 'KevinSpaghetti/accuracyk',
 'NCSOFT/harim_plus',
 'NikitaMartynov/spell-check-metric',
 'NimaBoscarino/weat',
 'Ochiroo/rouge_mn',
 'Vertaix/vendiscore',
 'Viona/infolm',
 'Vlasta/pr_auc',
 'abdusah/aradiawer',
 'abidlabs/mean_iou',
 'abidlabs/me

In [8]:
load_metric('accuracy')

<ipython-input-8-180ef0c0d6c3>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_metric('accuracy')


Metric(name: "accuracy", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = datasets.load_metric("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
        {'accuracy': 0.5}

   

# load data

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [25]:
#for Thai-only dataset
#!gdown 1DTktDhCzaWH5ZqXxR1F5ECCw81KxalIW

#for Thai-EN dataset
#!gdown 16fQtQeb7XMWQHKxA8aKTX5nLWq-vKwco

#for Thai-ZH dataset
#!gdown 1gsAZt2Qap257eYGDJvcgclBN4Rujc_NH

#for Thai-EN-ZH dataset
!gdown 1IoZ9R549JYtipOqqbpF3RAU3aS4Ne4EZ

#for Thai test set
!gdown 1Bfq_RyzGTvEj4sgVi5uH_4nS7di2jwzC

Downloading...
From: https://drive.google.com/uc?id=1DTktDhCzaWH5ZqXxR1F5ECCw81KxalIW
To: /content/THtwitter_NLI.csv
100% 492k/492k [00:00<00:00, 133MB/s]


In [11]:
#for Thai-only dataset
#df = pd.read_csv('THtwitter_NLI.csv')

#for Thai-EN dataset
#df = pd.read_csv('MULTI_th_en_NLI.csv')

#for Thai-ZH dataset
#df = pd.read_csv('MULTI_th_zh_NLI.csv')

#for Thai-EN-ZH dataset
df = pd.read_csv('MULTItwitter_NLI_train.csv')

#for Thai test set
df_test = pd.read_csv('MULTItwitter_NLI_test.csv')

In [12]:
#แปลง label เป็น index
labels = list(set(df['label'].to_list()))
label_to_idx = {label:i for i,label in enumerate(labels)}
df['label'] = df['label'].apply(lambda x: label_to_idx[x])
df_test['label'] = df_test['label'].apply(lambda x: label_to_idx[x])

label_list = [0] * len(label_to_idx)
for label in label_to_idx:
  label_list[label_to_idx[label]] = label 

In [13]:
print(label_to_idx)

{'contradiction': 0, 'neutral': 1, 'entailment': 2}


In [14]:
cls_token = tokenizer.cls_token
sep_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(cls_token, sep_token, pad_token, unk_token)
print(tokenizer.cls_token_id, tokenizer.sep_token_id, tokenizer.pad_token_id,  tokenizer.unk_token_id)

[CLS] [SEP] [PAD] [UNK]
101 102 0 100


In [15]:
#split train, dev, test
from sklearn.model_selection import train_test_split

df_train, df_dev = train_test_split(df, train_size=0.8)

len(df_train), len(df_dev), len(df_test)

(4064, 1016, 200)

# tokenize data


In [16]:
d = DatasetDict({'train': Dataset.from_pandas(df_train), 
                 'dev': Dataset.from_pandas(df_dev),
                 'test': Dataset.from_pandas(df_test)})

d['train'] = d['train'].remove_columns(['__index_level_0__'])
d['dev'] = d['dev'].remove_columns(['__index_level_0__'])

d

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 4064
    })
    dev: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 1016
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 200
    })
})

In [17]:
#tokenizer 
def tokenize(examples):
    tokenized_inputs = tokenizer(examples['premise'], 
                                 examples['hypothesis'], 
                                 add_special_tokens = True,
                                 truncation=True,
                                 is_split_into_words=False,
                                 max_length=250)
                                 #pad_to_max_length = True)
    return tokenized_inputs

tokenized_datasets = d.map(tokenize, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [18]:
print(tokenized_datasets['train'][0])

{'premise': 'The ugly is to bring depression to the pity. Causing people who are really depressed', 'hypothesis': 'คนส่วนมากสงสารคนที่เป็นโรคซึมเศร้า', 'label': 1, 'input_ids': [101, 10117, 11678, 10454, 10124, 10114, 26092, 51750, 10114, 10105, 55277, 10157, 119, 26037, 95179, 11426, 10479, 10301, 30181, 10104, 96616, 102, 25915, 28767, 31904, 73179, 66845, 18427, 28767, 19197, 28767, 52780, 31256, 16000, 18203, 54699, 111433, 22765, 31256, 91692, 111428, 17405, 111431, 50443, 22765, 43102, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [19]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4064
    })
    dev: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1016
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 200
    })
})

In [20]:
#drop column ที่ไม่ใช่ทิ้ง
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer,
                                        padding=True, pad_to_multiple_of=8)

tokenized_datasets = tokenized_datasets.remove_columns(["premise"])
tokenized_datasets = tokenized_datasets.remove_columns(["hypothesis"])
#tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['label', 'input_ids', 'token_type_ids', 'attention_mask']

# train model

In [21]:
args = TrainingArguments(
    output_dir='cross-lingual-Thai-NLI', 
    learning_rate = 8e-6,
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,
    num_train_epochs=15, 
    load_best_model_at_end=True, 
    metric_for_best_model='accuracy',
    evaluation_strategy='steps', 
    eval_steps=500, 
    #save_strategy='epoch'    
)

#metric.compute(predictions=preds, references=predictions.label_ids)
import numpy as np

def compute_metrics(eval_preds):
  metric = load_metric("accuracy")
  logits, labels = eval_preds #logists = probability แต่ละ label
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

#train model 
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4064
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3810
  Number of trainable parameters = 177855747
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
500,1.044500,0.963605,0.537402
1000,0.771500,0.889444,0.592520
1500,0.489800,0.858616,0.673228
2000,0.295700,0.878631,0.733268
2500,0.183000,0.992697,0.746063
3000,0.115500,1.159010,0.758858
3500,0.082400,1.195157,0.768701


***** Running Evaluation *****
  Num examples = 1016
  Batch size = 16
Saving model checkpoint to cross-lingual-Thai-NLI/checkpoint-500
Configuration saved in cross-lingual-Thai-NLI/checkpoint-500/config.json
Model weights saved in cross-lingual-Thai-NLI/checkpoint-500/pytorch_model.bin
tokenizer config file saved in cross-lingual-Thai-NLI/checkpoint-500/tokenizer_config.json
Special tokens file saved in cross-lingual-Thai-NLI/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1016
  Batch size = 16
Saving model checkpoint to cross-lingual-Thai-NLI/checkpoint-1000
Configuration saved in cross-lingual-Thai-NLI/checkpoint-1000/config.json
Model weights saved in cross-lingual-Thai-NLI/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in cross-lingual-Thai-NLI/checkpoint-1000/tokenizer_config.json
Special tokens file saved in cross-lingual-Thai-NLI/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1016
  

TrainOutput(global_step=3810, training_loss=0.39700823305785815, metrics={'train_runtime': 2245.0169, 'train_samples_per_second': 27.153, 'train_steps_per_second': 1.697, 'total_flos': 5796257035838976.0, 'train_loss': 0.39700823305785815, 'epoch': 15.0})

In [22]:
predictions = trainer.predict(tokenized_datasets['test'])
preds = np.argmax(predictions.predictions, axis=-1)

***** Running Prediction *****
  Num examples = 200
  Batch size = 16


In [23]:
predicted_labels = [labels[x] for x in preds]
#gold = [labels[x] for x in predictions.label_ids]
gold = [labels[x] for x in df_test['label']]

print(predicted_labels)
print(gold)

['entailment', 'entailment', 'entailment', 'neutral', 'neutral', 'entailment', 'contradiction', 'neutral', 'contradiction', 'contradiction', 'neutral', 'entailment', 'neutral', 'contradiction', 'neutral', 'contradiction', 'neutral', 'neutral', 'contradiction', 'contradiction', 'contradiction', 'contradiction', 'contradiction', 'contradiction', 'entailment', 'entailment', 'neutral', 'contradiction', 'neutral', 'neutral', 'contradiction', 'neutral', 'contradiction', 'entailment', 'neutral', 'entailment', 'neutral', 'entailment', 'contradiction', 'contradiction', 'neutral', 'entailment', 'contradiction', 'entailment', 'entailment', 'entailment', 'neutral', 'neutral', 'entailment', 'contradiction', 'contradiction', 'entailment', 'neutral', 'entailment', 'contradiction', 'contradiction', 'neutral', 'contradiction', 'neutral', 'neutral', 'contradiction', 'neutral', 'neutral', 'contradiction', 'entailment', 'neutral', 'entailment', 'neutral', 'entailment', 'entailment', 'contradiction', 'cont

In [24]:
from sklearn.metrics import classification_report
print(classification_report(gold, predicted_labels))

               precision    recall  f1-score   support

contradiction       0.92      0.91      0.92        66
   entailment       0.84      0.87      0.85        67
      neutral       0.89      0.88      0.89        67

     accuracy                           0.89       200
    macro avg       0.89      0.89      0.89       200
 weighted avg       0.89      0.89      0.89       200

